In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 2장 데이터읽기

In [ ]:
# 데이터 읽기
# 사용자 데이터 u.user 파일을 DataFrame으로 읽기
import os
import pandas as pd
base_src = '/content/drive/MyDrive/Colab Notebooks/인프런 강의/추천시스템/Python을 이용한 개인화 추천시스템'
u_user_src = os.path.join(base_src,'u.user')
u_cols = ['user_id','age','sex','occupation','zip_code']
users = pd.read_csv(u_user_src,sep='|',names= u_cols, encoding='latin-1')
users = users.set_index('user_id')
users

In [ ]:
u_item_src = os.path.join(base_src,'u.item')
i_cols = ['movie_id','title','releasedate','video release date'
,'IMDB URL','unknown','Action','Adventure','Animation'
,'Children\'s','Comedy','Crime','Documentary'
,'Drama','Fantasy','FilmNoir','Horror','Musical','Mystery','Romance'
,'Sci-Fi','Thriller','War','Western']
movies = pd.read_csv(u_item_src,sep='|',names= i_cols, encoding='latin-1')
movies = movies.set_index('movie_id')
movies

In [12]:
u_data_src = os.path.join(base_src,'u.data')
r_cols = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(u_data_src,sep='\t',names= r_cols, encoding='latin-1')
ratings = ratings.set_index('user_id')
ratings

,movie_id,rating,timestamp
user_id,,,
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596
...,...,...,...
880,476,3,880175444
716,204,5,879795543
276,1090,1,874795795


# 2장 인기 제품 추천

In [16]:
def recom_movie(n_items):
  # groupby['movie_id'] : 영화 id 코드로 묶기
  # 묶은뒤 rating 평균내기
  movie_mean = ratings.groupby(['movie_id'])['rating'].mean()
  # 평균된 영화평점기준으로 내림차순정렬 n개 뽑기
  movie_sort = movie_mean.sort_values(ascending=False)[:n_items]

  # movie_sort.index : n개 까지의 영화 index값만 가져오기


  recom_movies = movies.loc[movie_sort.index]
  recommendations = recom_movies['title']
  return recommendations

recom_movie(5)

movie_id
814                         Great Day in Harlem, A (1994)
1599                        Someone Else's America (1995)
1201           Marlene Dietrich: Shadow and Light (1996) 
1122                       They Made Me a Criminal (1939)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

# 2장 추천 시스템의 정확도 측정

In [54]:
import numpy as np

def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true)-np.array(y_pred))**2))

# 정확도 계산
rmse = []
movie_mean = ratings.groupby(['movie_id'])['rating'].mean()

for user in set(ratings.index):
  y_true = ratings.loc[user]['rating']
  # best-seller 방식으로
  y_pred = movie_mean[ratings.loc[user]['movie_id']]
  accuracy = RMSE(y_true,y_pred)
  rmse.append(accuracy)

print(np.mean(rmse))

0.7989582428029239


# 2-4 사용자 집단별 추천

In [28]:
# 데이터 읽기
# 사용자 데이터 u.user 파일을 DataFrame으로 읽기
import os
import pandas as pd
base_src = '/content/drive/MyDrive/Colab Notebooks/인프런 강의/추천시스템/Python을 이용한 개인화 추천시스템'
u_user_src = os.path.join(base_src,'u.user')
u_cols = ['user_id','age','sex','occupation','zip_code']
users = pd.read_csv(u_user_src,sep='|',names= u_cols, encoding='latin-1')

u_item_src = os.path.join(base_src,'u.item')
i_cols = ['movie_id','title','releasedate','video release date'
,'IMDB URL','unknown','Action','Adventure','Animation'
,'Children\'s','Comedy','Crime','Documentary'
,'Drama','Fantasy','FilmNoir','Horror','Musical','Mystery','Romance'
,'Sci-Fi','Thriller','War','Western']
movies = pd.read_csv(u_item_src,sep='|',names= i_cols, encoding='latin-1')

u_data_src = os.path.join(base_src,'u.data')
r_cols = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(u_data_src,sep='\t',names= r_cols, encoding='latin-1')

# ratings DataFrame에서 timestamp 제거
ratings = ratings.drop('timestamp',axis=1)
movies = movies[['movie_id','title']]

In [53]:
# 데이터 train, test set 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25, stratify=y)

# 정확도 RMSE 계산
def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true)-np.array(y_pred))**2))

# 모델별로 RMSE 계산
def score(model):
  id_pairs = zip(x_test['user_id'],x_test['movie_id'])
  y_pred = np.array([model(user,movie) for (user,movie) in id_pairs])
  y_true = np.array(x_test['rating'])
  return RMSE(y_true,y_pred)

# best_seller 함수를 이용한 정확도 계산
train_mean = x_train.groupby(['movie_id'])['rating'].mean()

def best_seller(user_id,movie_id):
  try:
    rating = train_mean[movie_id]

  except:
    rating = 3.0
  return rating

score(best_seller)

1.022820298848175

In [58]:
# 성별에 따른 예측값 계산
merged_ratings = pd.merge(x_train,users)

users = users.set_index('user_id')

g_mean = merged_ratings[['movie_id','sex','rating']].groupby(['movie_id','sex'])['rating'].mean()
g_mean

rating_matrix = x_train.pivot(index='user_id',
                              columns='movie_id',
                              values='rating')

In [62]:
# Gender 기준 추천
def cf_gender(user_id, movie_id):
  if movie_id in rating_matrix.columns:
    gender = users.loc[user_id]['sex']
    if gender in g_mean[movie_id].index:
      gender_rating = g_mean[movie_id][gender]
    else: # 없을경우
      gender_rating = 3.0
  else: # movie_id가없으면
    gender_rating = 3.0
  return gender_rating
score(cf_gender)

1.0308964836850518